## Imports

In [ ]:
import os
import re
import sqlite3
from uuid import uuid4

import pandas as pd
import requests

import chromadb
from langchain_ollama import OllamaEmbeddings
from langchain.chat_models import init_chat_model
from langchain_core.messages import AIMessage
from langgraph.checkpoint.memory import InMemorySaver
from langgraph.prebuilt import create_react_agent

client = chromadb.Client()
embeddings = OllamaEmbeddings(model="granite-embedding:278m")

## Pré-processamento

### Criação da base em SQL

In [ ]:
df = pd.read_json("data/dados-produtos.json")
conn = sqlite3.connect("produtos.db", check_same_thread=False)
df.to_sql("produtos", conn, if_exists="replace", index=False)

### Criação do banco vetorial com nomes dos produtos

In [ ]:
catalog_collection = client.create_collection(name="catalog")
for i, row in df.iterrows():
    catalog_collection.add(
        ids=[str(uuid4())],
        embeddings=embeddings.embed_query(row['description']),
        documents=[row['description']],
        metadatas=[{"productId": row['productId'], "title": row['title']}]
    )

In [ ]:
input_query = "calça de academia pra mulher"

results = catalog_collection.query(
  query_embeddings=embeddings.embed_query(input_query),
  n_results=5,
)
results

### Criação do banco vetorial do SAC

In [ ]:
collection = client.create_collection(name="sac")

with open('data/dados-sac.md', 'r', encoding='utf-8') as f:
    sac = f.read()

def split_faq_by_question_and_sentences(text):
    pattern = r"# (.*\?)"
    matches = list(re.finditer(pattern, text))
    chunks = []

    for i, match in enumerate(matches):
        start = match.end()
        end = matches[i + 1].start() if i + 1 < len(matches) else len(text)

        question = match.group(1).strip()
        answer_block = text[start:end].strip()

        subchunks = re.split(r"\.\s*\n", answer_block)
        subchunks = [chunk.strip() for chunk in subchunks if chunk.strip()]

        for j, sub in enumerate(subchunks):
            chunk_text = f"Pergunta: {question}\nResposta: {sub}."
            chunks.append({
                "text": chunk_text,
                "metadata": {
                    "question": question,
                    "chunk_id": f"{i}-{j}"
                }
            })

    return chunks

perguntas_chunks = split_faq_by_question_and_sentences(sac)
perguntas_chunks = [i['text'] for i in perguntas_chunks]

uuids = [str(uuid4()) for _ in range(len(perguntas_chunks))]
for i, d in enumerate(perguntas_chunks):
    collection.add(
        ids=[uuids[i]],
        embeddings=embeddings.embed_query(d),
        documents=[d]
    )

## Criação das tools

In [ ]:
def search_sql(query):
    """Busca informações de produtos, suas características e preços no banco via SQLite. Considere o seguinte schema e uma pequena amostra da tabela 'produtos':
    [(0, 'index', 'BIGINT', 0, None, 0), (1, 'productId', 'BIGINT', 0, None, 0), (2, 'title', 'TEXT', 0, None, 0), (3, 'price', 'FLOAT', 0, None, 0), (4, 'image', 'TEXT', 0, None, 0), (5, 'description', 'TEXT', 0, None, 0)]

    [(1666, 2009094, 'camisa manga longa com bolso branco', 139.99, 'https://cea.vteximg.com.br/arquivos/ids/57197111/camisa-manga-longa-com-bolso-branco-7591834-Branco_1.jpg?v=638128914633000000', 'camisa manga longa bolso branco moda masculina roupas blusas 1 2 3 4 5 6 7'), (1667, 2017098, 'camisa manga curta com bolso branca', 119.99, 'https://cea.vteximg.com.br/arquivos/ids/53195213/camisa-manga-curta-com-bolso-branco-7602490-Branco_1.jpg?v=637801158434200000', 'camisa manga curta bolso branca moda masculina roupas blusas branco 3 4 5 2 1 6 7'), (1326, 2062137, 'Calça Jeans Feminina Super Skinny com Zíper na Barra Azul Médio', 119.99, 'https://cea.vteximg.com.br/arquivos/ids/52448505/Calca-Jeans-Feminina-Super-Skinny-com-Ziper-na-Barra-Azul-Medio-7936103-Azul_Medio_1.jpg?v=637774164382300000', 'calca jeans feminina super skinny ziper barra azul medio moda roupas calcas 40 46 36 38 44 42 48 34')]
    """
    try:
        return pd.read_sql_query(query, conn).to_string()
    except Exception as e:
        return f"Erro na consulta SQL: {str(e)}"
    
def search_sac(question):
    """Busca informações sobre o serviço de atendimento ao cliente."""
    embedded = embeddings.embed_query(question)
    results = collection.query(
        query_embeddings=[embedded],
        n_results=5
    )
    return results

def search_proper_nouns_catalog(question):
    """
    Use para filtrar a coluna 'description' na tabela 'produtos'. 
    O input pode ser a descrição do que o usuário precisa ou nome aproximado do produto, o output é o nome exato como consta na coluna 'description'.
    """
    embedded = embeddings.embed_query(question)
    results = catalog_collection.query(
        query_embeddings=[embedded],
        n_results=5
    )
    print(results['documents'][0])
    print(results['metadatas'][0])
    return results

## Criação do agente e configuração do modelo de chat

In [ ]:
model = init_chat_model(
    model="llama3.1",
    model_provider="ollama",
    temperature=0
)

checkpointer = InMemorySaver()

REACT_PROMPT = """
Você é um assistente especializado em responder perguntas sobre a C&A.

Você tem acesso a duas fontes principais de informação:
- As respostas para as perguntas mais frequentes dos clientes.
- O catálogo de produtos, incluindo descrições detalhadas e preços.

<instructions>
Ao receber uma pergunta do usuário:
	1.	Considere sempre o contexto da conversa antes de decidir qual ferramenta utilizar.
	2.	Avalie cuidadosamente se a informação encontrada é relevante e suficiente para responder com precisão.
	3.	Caso necessário, utilize mais de uma ferramenta para formular a resposta.
	4.	Organize suas respostas de forma clara, direta e útil para o usuário.
</instructions>
    
Pense passo a passo antes de responder, garantindo que sua resposta seja completa e alinhada à pergunta feita.
"""

agent_executor = create_react_agent(model=model, 
                                    tools=[search_sac, search_sql, search_proper_nouns_catalog], 
                                    prompt=REACT_PROMPT, 
                                    checkpointer=checkpointer)

def answer_question(question: str, thread_id: str = ""):
    config = {"configurable": {"thread_id": thread_id}}
    last_event = None
    input_tokens = 0
    output_tokens = 0

    for event in agent_executor.stream(
        {"messages": [{"role": "user", "content": question}]},
        stream_mode="values", config=config,
    ):
        event["messages"][-1].pretty_print()
        last_event = event            

    for message in last_event["messages"]:
        if isinstance(message, AIMessage):
            input_tokens += message.usage_metadata['input_tokens']
            output_tokens += message.usage_metadata['output_tokens']
    
    resposta = last_event["messages"][-1].content

    return {
        "query": question,
        "answer": resposta,
        "input_tokens": input_tokens,
        "output_tokens": output_tokens
    }

## Teste

Para testar a continuidade da conversa, basta repetir a função answer_question com o mesmo thread_id

In [ ]:
answer_question("Olá!", thread_id="teste-123")